In [1]:
import numpy as np
from scipy.special import erf
import os
import sys
import time
import copy
import logging
import matplotlib.pyplot as plt

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)



from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b

from RP_mechanisms.optim_RP_DP import compute_largest_l2
from LS_mechanisms.alt19_LS import ALT19LS_mech
from LS_mechanisms.optim_LS_DP import LS_fromoptim_RP_mech, OptimalLS_mech
from LS_mechanisms.LS_accuracy_estimator import LS_accuracy_estimator

In [2]:
logfile_path = os.path.join(log_dir, 'LS-cancer-default.log')
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(message)s",
    handlers=[
        logging.FileHandler(logfile_path),
        logging.StreamHandler()
    ]
)

def store_array_str(data_array):
    converted_str = np.array_str(data_array)
    converted_str = converted_str.replace(' ',',')
    return converted_str

file_X_path = os.path.join(project_dir, "Dataset", "cancer-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "cancer-LR-y.txt")

X = None
y = None
try:
    X, y = load_cancer_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    main_generate_cancer_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_cancer_dataset_matrices_for_LS()

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [4]:
def generate_default_configuration():
    kwargs = {
        'database': A,
        'r' : 300,
        'CI' : 0.95,
        'bootstrap_samples': 100, 
        'l2' : compute_largest_l2(A)
    }
    
    return kwargs

kwargs = generate_default_configuration()

In [5]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

7.917416411545128

In [6]:
mech = OptimalLS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:12:58,136 cost 2.5067 seconds


In [7]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:12:58,167 (bootstrap_result(lb_one_sided=np.float64(0.9943237091481387), lb_two_sided=np.float64(0.990857713523088), ub_one_sided=np.float64(1.0121422582057171), ub_two_sided=np.float64(1.0157612812312413)), bootstrap_result(lb_one_sided=np.float64(4.7182526806821885), lb_two_sided=np.float64(4.694999238027325), ub_one_sided=np.float64(5.099316157763447), ub_two_sided=np.float64(5.141578329101455)))
2025-10-28 20:12:58,168 cost 0.0244 seconds


In [8]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.find_minimal_sigma(epsilon, delta)

/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: overflow encountered in exp
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)
/home/wei402/Desktop/NDIS/src/RP_mechanisms/optim_RP.py:25: RuntimeWarning: invalid value encountered in scalar multiply
  exp_term = np.exp(epsilon - r / 2 * np.log(1 - leverage)) * (1 - leverage) ** (r / 2.0)


np.float64(6.850481817751286)

In [9]:
mech = LS_fromoptim_RP_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:13:02,341 cost 4.1573 seconds


In [10]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:13:02,376 (bootstrap_result(lb_one_sided=np.float64(0.9895514489581121), lb_two_sided=np.float64(0.9869657860472232), ub_one_sided=np.float64(1.0119871460842775), ub_two_sided=np.float64(1.015175516400725)), bootstrap_result(lb_one_sided=np.float64(4.746041152454643), lb_two_sided=np.float64(4.736801333411779), ub_one_sided=np.float64(5.137166982445114), ub_two_sided=np.float64(5.143190082995226)))
2025-10-28 20:13:02,377 cost 0.0235 seconds


In [11]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n

mech.compute_constant(epsilon, delta)

np.float64(20.317246800063643)

In [12]:
mech = ALT19LS_mech(kwargs)

epsilon = 1
delta = 1/mech.n
num_samples = 100

tic = time.perf_counter()
samples = mech._gen_samples(epsilon, delta, num_samples)
toc = time.perf_counter()
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:13:06,431 cost 4.0279 seconds


In [13]:
tic = time.perf_counter()
accuracy_estimator = LS_accuracy_estimator(kwargs)
boot_res = accuracy_estimator.estimate_square_error(samples)
toc = time.perf_counter()
logging.critical(boot_res)
logging.critical(f"cost {toc - tic:0.4f} seconds")

2025-10-28 20:13:06,461 (bootstrap_result(lb_one_sided=np.float64(1.007409094559596), lb_two_sided=np.float64(1.0039442467411834), ub_one_sided=np.float64(1.0295320995377746), ub_two_sided=np.float64(1.0314127085048712)), bootstrap_result(lb_one_sided=np.float64(4.5889236576838), lb_two_sided=np.float64(4.580661124845778), ub_one_sided=np.float64(4.920849816893964), ub_two_sided=np.float64(4.944371829986711)))
2025-10-28 20:13:06,462 cost 0.0248 seconds
